In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torch.autograd import Variable
import time

import joblib

In [2]:
cuda = torch.cuda.is_available()

In [3]:
train_data = f'../ISMP/dataset/trainset_v2_500_label4.jb'

In [4]:
def num_flat_features(x):
    size = x.size()[1:]  # all dimensions except the batch dimension
    num_features = 1
    for s in size:
        num_features *= s
    return num_features

In [5]:
X, y = joblib.load(train_data)

In [6]:
X = torch.tensor(X, dtype = torch.float)

In [7]:
cnn1d = nn.Conv1d(3, 5, kernel_size=5)
m = nn.MaxPool1d(5, stride=2)
lin = nn.Linear(246 * 5, 2, bias=False)

In [8]:
x = cnn1d(X)
x = m(x)
x = F.leaky_relu(x)
x = x.view(-1, num_flat_features(x))
x = lin(x)

In [9]:
x.shape

torch.Size([6900, 2])

# Data

In [10]:
class EQDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.transform = transform
        self.data_dir = data_dir
        train_x, train_y = joblib.load(data_dir)
        self.train_x, self.train_y = torch.tensor(train_x, dtype = torch.float), torch.tensor(train_y, dtype = torch.float)
        self.len = self.train_x.shape[0]
        
    def __len__(self):
        return self.len
    
    def __getitem__(self, idx):
        
        X = self.train_x[idx]
        y = self.train_y[idx]

        if self.transform:
            X = self.transform(X)
            
        return X, y

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(3, 5, kernel_size=5)
        self.maxP1 = nn.MaxPool1d(5, stride=2)
        self.lin = nn.Linear(246 * 5, 2, bias=False)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxP1(x)
        x = F.leaky_relu(x)
        x = x.view(-1, self.num_flat_features(x))
        x = self.lin(x)
        
        return x
    
    def forwardToHidden(self, x):
        x = self.conv1(x)
        x = self.maxP1(x)
        x = F.leaky_relu(x)
        x = x.view(-1, self.num_flat_features(x))
        
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [12]:
model = Net()

In [13]:
model

Net(
  (conv1): Conv1d(3, 5, kernel_size=(5,), stride=(1,))
  (maxP1): MaxPool1d(kernel_size=5, stride=2, padding=0, dilation=1, ceil_mode=False)
  (lin): Linear(in_features=1230, out_features=2, bias=False)
)

In [14]:
list(model.parameters())[0].shape

torch.Size([5, 3, 5])

In [15]:
list(model.parameters())[1].shape

torch.Size([5])

In [16]:
list(model.parameters())[2].shape

torch.Size([2, 1230])

In [17]:
class pseudoInverse(object):
    def __init__(self, params, C=1e-2, L =100):
        self.C = C
        self.L = L
        
        self.params = list(params)
        self.w = self.params[-1]
        self.w.data.fill_(0)
        self.cuda = self.w.is_cuda
        self.dimInput, self.dimOutput = self.w.data.size()
        
    def pseudoSparse(self, H, oneHotTarget):
        hth = torch.mm(H.t(), H) #[n_hidden, n_hidden]
        dimHidden = H.size()[1]
        I = Variable(torch.eye(dimHidden), requires_grad = False)
        if self.cuda:
            I = I.cuda()
        
        if self.L > 0.0:
            mu = torch.mean(H, dim = 0, keepdim = True) #[1, n_features]
            S = H - mu
            S = torch.mm(S.t(), S)
            self.B = Variable(torch.inverse(hth.data + self.C * (I.data + self.L * self.S)), requires_grad = False)
        else:
            self.B = Variable(torch.inverse(hth.data + self.C * I.data), requires_grad = False)
            
        w = torch.mm(self.B, H.t())
        #print(w)
        #print(oneHotTarget.reshape(-1, 2))
        w = torch.mm(w, oneHotTarget)
        self.w.data = w.t().data
    
    def pseudoCompress(self, H, oneHotTarget):
        hht = torch.mm(H, H.t()) #[n_features, n_features]
        numSample = H.size()[0]
        I = Variable(torch.eye(numSample), requires_grad = False)
        if self.cuda:
            I = I.cuda()
            
        self.B = Variable(torch.inverse(hht.data + self.C * I.data), requires_grad = False)
        w = torch.mm(H.t(), self.B)
        w = torch.mm(w, oneHotTarget)
        self.w.data = w.t().data
        
    def get_hidden(self, hidden):
        return {'sparse':self.pseudoSparse, 'compress':self.pseudoCompress}[hidden]
    
    
    def train(self, H, y, oneHot = True, hiddenType = 'sparse'):
        y = y.view(y.size(0),-1)
        
        if oneHot:
            oneHotTarget = self.oneHotVectorize(y)
            
        hidden = self.get_hidden(hiddenType)
        hidden(H, oneHotTarget)

        
    def oneHotVectorize(self,targets):
        oneHotTarget = torch.zeros(targets.size()[0], self.dimOutput)

        for i in range(targets.size()[0]):
            oneHotTarget[i][int(targets[i].item())] = 1

        if self.cuda:
            oneHotTarget=oneHotTarget.cuda()
            
        oneHotTarget = Variable(oneHotTarget, requires_grad=False)

        return oneHotTarget

In [18]:
train_dir = f'../ISMP/dataset/trainset_v2_500_label4.jb'
test_dir = f'../ISMP/dataset/testing_500_label4.jb'

train_datasets = EQDataset(train_dir)
test_datasets = EQDataset(test_dir)

X_train, y_train = train_datasets[:]
X_test, y_test = test_datasets[:]

In [19]:
model = Net()
if cuda:
    model.cuda()
optimizer= pseudoInverse(params=model.parameters(), C=0.1,L=0)

In [20]:
X_train, y_train = X_train.cuda(), y_train.cuda()
X_train, y_train = Variable(X_train, requires_grad=False), Variable(y_train, requires_grad=False)

hiddenOut = model.forwardToHidden(X_train)
optimizer.train(hiddenOut, y_train)
output = model.forward(X_train)
pred = output.data.max(1)[1]

print(pred.eq(y_train.data).cpu().sum())

tensor(6120)


In [21]:
y_train.shape

torch.Size([6900])

In [22]:
6255 / 6900

0.9065217391304348

# Test

In [23]:
X_test, y_test = X_test.cuda(), y_test.cuda()
X_train, y_test = Variable(X_test, requires_grad=False), Variable(y_test, requires_grad=False)

output = model.forward(X_test)
pred = output.data.max(1)[1]

print(pred.eq(y_test.data).cpu().sum())

tensor(1689)


In [24]:
y_test.shape

torch.Size([1751])

In [26]:
1689 / 1751

0.9645916619074815